<a href="https://colab.research.google.com/github/rishita3863/Performance-comparision-IaaS-and-PaaS/blob/main/CC_Internal_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install psutil requests


In [ ]:
import requests
import time
import psutil


In [ ]:
IAAS_URL = "https://httpbin.org/get"
PAAS_URL = "https://jsonplaceholder.typicode.com/posts/1"

In [ ]:
def benchmark_service(url, num_requests=10):
    response_times = []
    cpu_usages = []
    memory_usages = []

    for i in range(num_requests):
        # CPU and memory before request
        cpu_before = psutil.cpu_percent(interval=None)
        mem_before = psutil.virtual_memory().percent

        # Send request and measure response time
        start_time = time.time()
        response = requests.get(url)
        end_time = time.time()

        # CPU and memory after request
        cpu_after = psutil.cpu_percent(interval=None)
        mem_after = psutil.virtual_memory().percent

        # Calculate response time in ms
        resp_time = (end_time - start_time) * 1000

        # Save values
        response_times.append(resp_time)
        cpu_usages.append((cpu_before + cpu_after) / 2)
        memory_usages.append((mem_before + mem_after) / 2)

        print(f"Request {i+1}: {resp_time:.2f} ms | CPU {cpu_usages[-1]:.2f}% | Memory {memory_usages[-1]:.2f}%")

    # Average values
    avg_response = sum(response_times) / len(response_times)
    avg_cpu = sum(cpu_usages) / len(cpu_usages)
    avg_memory = sum(memory_usages) / len(memory_usages)

    return avg_response, avg_cpu, avg_memory


In [ ]:
print("Benchmarking IaaS...")
iaas_results = benchmark_service(IAAS_URL, num_requests=10)

print("\nBenchmarking PaaS...")
paas_results = benchmark_service(PAAS_URL, num_requests=10)


Benchmarking IaaS...
Request 1: 509.77 ms | CPU 10.95% | Memory 8.10%
Request 2: 1292.55 ms | CPU 1.35% | Memory 8.10%
Request 3: 2328.99 ms | CPU 1.30% | Memory 8.10%
Request 4: 1748.32 ms | CPU 1.45% | Memory 8.10%
Request 5: 1065.42 ms | CPU 1.20% | Memory 8.10%
Request 6: 1262.32 ms | CPU 1.00% | Memory 8.10%
Request 7: 2765.29 ms | CPU 16.60% | Memory 8.15%
Request 8: 988.57 ms | CPU 1.50% | Memory 8.20%
Request 9: 784.19 ms | CPU 1.25% | Memory 8.20%
Request 10: 1194.04 ms | CPU 1.90% | Memory 8.20%

Benchmarking PaaS...
Request 1: 25.93 ms | CPU 10.00% | Memory 8.20%
Request 2: 20.11 ms | CPU 0.00% | Memory 8.20%
Request 3: 25.41 ms | CPU 10.00% | Memory 8.20%
Request 4: 20.34 ms | CPU 10.00% | Memory 8.20%
Request 5: 17.42 ms | CPU 0.00% | Memory 8.20%
Request 6: 18.29 ms | CPU 12.50% | Memory 8.20%
Request 7: 19.05 ms | CPU 0.00% | Memory 8.20%
Request 8: 18.82 ms | CPU 12.50% | Memory 8.20%
Request 9: 22.57 ms | CPU 0.00% | Memory 8.20%
Request 10: 23.69 ms | CPU 12.50% | Mem

In [ ]:
print("\n=== Final Comparison ===")
print(f"IaaS -> Response: {iaas_results[0]:.2f} ms | CPU: {iaas_results[1]:.2f}% | Memory: {iaas_results[2]:.2f}%")
print(f"PaaS -> Response: {paas_results[0]:.2f} ms | CPU: {paas_results[1]:.2f}% | Memory: {paas_results[2]:.2f}%")



=== Final Comparison ===
IaaS -> Response: 1393.95 ms | CPU: 3.85% | Memory: 8.13%
PaaS -> Response: 21.16 ms | CPU: 6.75% | Memory: 8.20%


In [ ]:
!pip install plotly


In [ ]:
results = {
    "IaaS": {
        "Response Time (ms)": iaas_results[0],
        "CPU Usage (%)": iaas_results[1],
        "Memory Usage (%)": iaas_results[2]
    },
    "PaaS": {
        "Response Time (ms)": paas_results[0],
        "CPU Usage (%)": paas_results[1],
        "Memory Usage (%)": paas_results[2]
    }
}


In [ ]:
import plotly.graph_objects as go

# Prepare data
categories = ["Response Time (ms)", "CPU Usage (%)", "Memory Usage (%)"]
iaas_values = [results["IaaS"][cat] for cat in categories]
paas_values = [results["PaaS"][cat] for cat in categories]

# Create bar chart
fig = go.Figure(data=[
    go.Bar(name="IaaS", x=categories, y=iaas_values, marker_color="blue"),
    go.Bar(name="PaaS", x=categories, y=paas_values, marker_color="green")
])

# Update layout
fig.update_layout(
    title="IaaS vs PaaS Performance Comparison",
    xaxis_title="Metrics",
    yaxis_title="Values",
    barmode="group",
    template="plotly_white"
)

# Show in Colab
fig.show()


In [ ]:
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=("Bar Comparison", "Results Table"),
    specs=[[{"type": "bar"}],
           [{"type": "table"}]]
)

# Add bar chart
fig.add_trace(go.Bar(name="IaaS", x=categories, y=iaas_values, marker_color="blue"), row=1, col=1)
fig.add_trace(go.Bar(name="PaaS", x=categories, y=paas_values, marker_color="green"), row=1, col=1)

# Add table
fig.add_trace(go.Table(
    header=dict(values=["Metric", "IaaS", "PaaS"], fill_color="lightgrey", align="center"),
    cells=dict(values=[categories, iaas_values, paas_values], align="center")
), row=2, col=1)

# Layout
fig.update_layout(title="IaaS vs PaaS Dashboard", barmode="group", template="plotly_white")
fig.show()
